In [18]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [19]:
def load_dataset():
    """
    Load training and test sets
    """

    train = pd.read_csv("./twitter_dataset/train.csv")
    dev = pd.read_csv("./twitter_dataset/dev.csv")
    
    return train, dev

In [20]:
train, dev = load_dataset()

In [51]:
train

,Unnamed: 0,id,text,label,label_text,Y_hard,Embedding,Tokenized Text,Y_soft
0,23738,20c2ecb64f,Had a nice visit last night from a boy with vo...,2,positive,"[0.0, 0.0, 1.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['Had', 'a', 'nice', 'visit', 'last', 'night',...","[0.014292021282017231, 0.10583829134702682, 0...."
1,26930,2625f6b91b,never mind it`s closed sorry Miley,0,negative,"[1.0, 0.0, 0.0]","[array([ 2.0315e-01, -1.8103e-01, 2.8270e-01,...","['never', 'mind', 'it', '`s', 'closed', 'sorry...","[0.6122762560844421, 0.3607501685619354, 0.026..."
2,9119,6f56e42577,I won`t leave you alone until you accept my a...,2,positive,"[0.0, 0.0, 1.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['I', 'won', '`t', 'leave', 'you', 'alone', 'u...","[0.08997537940740585, 0.7731336951255798, 0.13..."
3,25447,0c61206a16,not gonna ask what a boom boom is! morning.,1,neutral,"[0.0, 1.0, 0.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['not', 'gonna', 'ask', 'what', 'a', 'boom', '...","[0.315542608499527, 0.5471459031105042, 0.1373..."
4,25158,203a5f037e,Btw that song is you`ll always find ur way bac...,1,neutral,"[0.0, 1.0, 0.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['Btw', 'that', 'song', 'is', 'you', '`ll', 'a...","[0.006457450799643993, 0.3284298777580261, 0.6..."
...,...,...,...,...,...,...,...,...,...
19231,21575,442a264f77,STAR TREK WAS PURE AWESOME! LOVE IT!!! <3333 ...,2,positive,"[0.0, 0.0, 1.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['STAR', 'TREK', 'WAS', 'PURE', 'AWESOME', '!'...","[0.0031628634314984083, 0.005445864982903004, ..."
19232,5390,c4994fd1f4,"Will be going to Indiana Baptist Sunday, Pray ...",1,neutral,"[0.0, 1.0, 0.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['Will', 'be', 'going', 'to', 'Indiana', 'Bapt...","[0.013718850910663605, 0.5312982201576233, 0.4..."
19233,860,c0360fd4e9,is sitting thru the boring bits in Titanic wai...,1,neutral,"[0.0, 1.0, 0.0]","[array([-3.9621e-04, 4.5670e-01, 3.3890e-01,...","['is', 'sitting', 'thru', 'the', 'boring', 'bi...","[0.4319244623184204, 0.4801938831806183, 0.087..."
19234,15795,4c7e112f86,Missed the play,0,negative,"[1.0, 0.0, 0.0]","[array([ 9.5152e-02, 3.7024e-01, 5.4291e-01,...","['Missed', 'the', 'play']","[0.278874009847641, 0.655164897441864, 0.06596..."


In [44]:
# Assuming we have train and test sets
Xmat_train = train["text"]
Y_train = train["Y_hard"]

Xmat_dev = dev["text"]
Y_dev = dev["Y_hard"]

# Handle missing values
Xmat_train.fillna('', inplace=True)
Xmat_dev.fillna('', inplace=True)

# Create a Bag of Words representation using the training data *only*
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(Xmat_train)
X_dev_bow = vectorizer.transform(Xmat_dev)

In [48]:
# Train a Logistic Regression model
baseline_bow = LogisticRegression(max_iter=5000, multi_class='auto', solver='lbfgs')
baseline_bow.fit(X_train_bow, Y_train)

# Make predictions
Y_pred_train = baseline_bow.predict(X_train_bow)
Y_pred_dev = baseline_bow.predict(X_dev_bow)

# Evaluate the model
train_accuracy = accuracy_score(Y_train, Y_pred_train)
dev_accuracy = accuracy_score(Y_dev, Y_pred_dev)

print(f"Train accuracy: {train_accuracy}")
print(f"Dev accuracy: {dev_accuracy}")


Train accuracy: 0.9156789353295903
Dev accuracy: 0.6895087932080048
